In [ ]:
import pandas as pd
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday
from pandas.tseries.offsets import DateOffset
from dateutil.relativedelta import MO
from prophet import Prophet
import numpy as np
import matplotlib.pyplot as plt
from pmdarima import auto_arima
import os

In [2]:
# Lectura de serie de tiempo
os.chdir("/home/cesar/Documentos/Banco_Base")
data = pd.read_csv('./Ejercicio_2/datasets/tipo_cambio.csv')
data['Fecha'] = pd.to_datetime(data['Fecha'],format="%d/%m/%Y")

In [3]:
# Lectura de precio de crudo de petroleo
petroleo = pd.read_csv('./Ejercicio_2/datasets/petroleo.csv')
petroleo['precio_dolares'] = petroleo['precio_dolares'].replace('N/E', np.nan)
petroleo["precio_dolares"] = pd.to_numeric(petroleo["precio_dolares"])
petroleo['precio_dolares'] = petroleo['precio_dolares'].interpolate(method='linear')
petroleo['Fecha'] = pd.to_datetime(petroleo['Fecha'],format="%d/%m/%Y")
# Merge de datos
data2 = pd.merge(data, petroleo, on='Fecha', how='left')
# Prophet usa columnas 'ds' (fecha) y 'y' (valor)
data2.columns = ['ds', 'y', 'precio_petroleo']
data2_valido = data2[data2['ds'] <= pd.to_datetime('2024-09-30')]
data2_valido = data2_valido[data2_valido['ds'] > pd.to_datetime('2020-01-01')]
data2_no_valido= data2[data2['ds'] > pd.to_datetime('2024-09-30')]


In [ ]:
### Crear un calendario de días festivos en México
class DiasFestivos(AbstractHolidayCalendar):
    rules = [
        Holiday('Año Nuevo', month=1, day=1),
        # Primer lunes de febrero
        Holiday('Día de la Constitución', month=2, day=1, offset=DateOffset(weekday=MO(1))),
        # Tercer lunes de marzo
        Holiday('Natalicio de Benito Juárez', month=3, day=1, offset=DateOffset(weekday=MO(3))),
        Holiday('Día del Trabajo', month=5, day=1),
        Holiday('Día de la Independencia', month=9, day=16),
        # Toma protesta
        Holiday('Toma protesta', year=2024, month=10, day=1),
        # Tercer lunes de noviembre
        Holiday('Día de la Revolución', month=11, day=1, offset=DateOffset(weekday=MO(3))),
        Holiday('Navidad', month=12, day=25),
    ]

# Generar las fechas festivas
calendar = DiasFestivos()
festivos_mx = calendar.holidays(start='2014-01-02', end='2024-12-31')
festivos_mx


In [ ]:
###### Ajuste de modelo Prophet
# Modelo sin variables externas (BaseModel)
model = Prophet(
    holidays=pd.DataFrame({'holiday': 'Festivos_MX', 'ds': festivos_mx}),
    daily_seasonality=False,
    weekly_seasonality=True,  
    yearly_seasonality=True 
)
model.fit(data2_valido.drop(['precio_petroleo'], axis=1))

# Modelo con variables externas
model2 = Prophet(
    holidays=pd.DataFrame({'holiday': 'Festivos_MX', 'ds': festivos_mx}),
    daily_seasonality=False,
    weekly_seasonality=True,  
    yearly_seasonality=True 
)
model2.add_regressor('precio_petroleo')
model2.fit(data2_valido)

In [ ]:
### Generar el pronóstico y MAPE para model 1
# Dataframe de pronostico
future = model.make_future_dataframe(periods=58)  # Pronosticar hasta 2024-11-27
future = future[~future['ds'].dt.weekday.isin([5, 6])]  # Excluir fines de semana
future = future[~future['ds'].isin(festivos_mx)]       # Excluir días festivos
# Pronostico
forecast = model.predict(future)
df_forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
df_test = df_forecast.tail(40)
# Calculo del mape
pe = abs(np.array(df_test['yhat']) - np.array(data2_no_valido['y'])) / np.array(data2_no_valido['y'])
mape = np.mean(pe)
print(mape)
print(df_test)


In [ ]:
### Generar el pronóstico y MAPE para model 2
# Dataframe de pronostico
future2 = model2.make_future_dataframe(periods=58)  # Pronosticar hasta 2024-11-27
future2 = future2[~future2['ds'].dt.weekday.isin([5, 6])]  # Excluir fines de semana
future2 = future2[~future2['ds'].isin(festivos_mx)]       # Excluir días festivos
future2 = pd.merge(future2, data2[['ds','precio_petroleo']], on='ds', how='left')

# Pronostico
forecast2 = model2.predict(future2)
df_forecast2 = forecast2[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
df_test2 = df_forecast2.tail(40)
# Calculo del mape
pe = abs(np.array(df_test2['yhat']) - np.array(data2_no_valido['y'])) / np.array(data2_no_valido['y'])
mape2 = np.mean(pe)
print(mape2)
print(df_test2)

In [ ]:
## Plot model 1
fig = model.plot(forecast)
# Agregar una línea vertical para marcar el inicio del horizonte de pronóstico
horizon_start = future['ds'].iloc[len(data2_valido)]  # Primera fecha del horizonte de pronóstico
plt.axvline(x=horizon_start, color='red', linestyle='--', label='Inicio de pronóstico')
ultimo_anio = data2_valido['ds'].max() - pd.DateOffset(years=1)
plt.xlim(ultimo_anio, forecast['ds'].max())  # Limitar el eje X

# Agregar una leyenda
plt.legend()
# plt.savefig('./Ejercicio_2/figures/model_1_zoom.png')
# Mostrar el gráfico
plt.show()

In [ ]:
## Plot model 2
fig = model2.plot(forecast2)
# Agregar una línea vertical para marcar el inicio del horizonte de pronóstico
horizon_start = future2['ds'].iloc[len(data2_valido)]  # Primera fecha del horizonte de pronóstico
plt.axvline(x=horizon_start, color='red', linestyle='--', label='Inicio de pronóstico')
ultimo_anio = data2_valido['ds'].max() - pd.DateOffset(years=1)
plt.xlim(ultimo_anio, forecast2['ds'].max())  # Limitar el eje X

# Agregar una leyenda
plt.legend()
# plt.savefig('./Ejercicio_2/figures/model_2_zoom.png')
# Mostrar el gráfico
plt.show()